In [21]:

import os
import pandas as pd
import numpy as np
import shutil
import arcpy
from arcpy import env
from arcpy.management import *
from arcpy.sa import *
from arcpy.da import *
from arcpy.conversion import *

In [22]:

# 获取栅格的顶角坐标
def get_raster_extent(raster):
    desc = arcpy.Describe(raster)
    extent = desc.extent
    return [extent.XMin,extent.YMax]

# 获取栅格的像元数量
def get_raster_size(raster):
    desc = arcpy.Describe(raster)
    return [desc.width,desc.height]

# 调整栅格范围一致
def reshape_raster(stand_raster,in_raster):
    """stand_raster:参考栅格,in_raster:待调整栅格"""
    stand_raster = Raster(stand_raster)
    in_raster= Raster(in_raster)
    with arcpy.EnvManager(snapRaster=stand_raster, extent="MAXOF", cellSize=stand_raster):
        output_raster = Con(stand_raster>0,in_raster)
    return output_raster
# 调整栅格size一致
def reshape_raster_size(stand_raster,in_raster,cell_size):
    """stand_raster:参考栅格,in_raster:待调整栅格,cell_size:像元大小"""
    stand_raster = Raster(stand_raster)
    env.extent = stand_raster
    in_raster= Raster(in_raster)
    stander_array = arcpy.RasterToNumPyArray(stand_raster)
    in_array = arcpy.RasterToNumPyArray(in_raster)
    result_array = resize_arrays(stander_array,in_array,0)[1]
    # array 转 raster
    return mask_raster(result_array,stand_raster,cell_size)
    

In [23]:
# 输出路径
out_path = r"D:\ArcgisData\basedata\basetable\size_30"

In [24]:
# 读取并检查栅格形状尺寸是否一致
raster_path = r"D:\ArcgisData\basedata\basetrain_30m.gdb"
env.workspace = raster_path
raster_list = arcpy.ListRasters()

In [25]:
# 导出栅格数组
for one_raster in raster_list:
    temp_index = get_raster_extent(one_raster)
    temp_size = get_raster_size(one_raster)
    print(one_raster)
    print(temp_index,temp_size)
    temp_array = pd.DataFrame(arcpy.RasterToNumPyArray(one_raster))
    temp_array.to_csv(os.path.join(out_path,f"{one_raster}.csv"))

DEM
[397387.5, 3153427.5] [1849, 2481]
TWI5
[397387.5, 3153427.5] [1849, 2481]
TPI201
[397387.5, 3153427.5] [1849, 2481]
TPI101
[397387.5, 3153427.5] [1849, 2481]
TPI11
[397387.5, 3153427.5] [1849, 2481]
TPI3
[397387.5, 3153427.5] [1849, 2481]
TMP
[397387.5, 3153427.5] [1849, 2481]
SOILQS
[397387.5, 3153427.5] [1849, 2481]
SLOP
[397387.5, 3153427.5] [1849, 2481]
PRE
[397387.5, 3153427.5] [1849, 2481]
NIGTH
[397387.5, 3153427.5] [1849, 2481]
NDVI
[397387.5, 3153427.5] [1849, 2481]
CUR
[397387.5, 3153427.5] [1849, 2481]
ASP
[397387.5, 3153427.5] [1849, 2481]
PLCUR
[397387.5, 3153427.5] [1849, 2481]
POCUR
[397387.5, 3153427.5] [1849, 2481]
SOM
[397387.5, 3153427.5] [1849, 2481]
PH
[397387.5, 3153427.5] [1849, 2481]
LON
[397387.5, 3153427.5] [1849, 2481]
LAT
[397387.5, 3153427.5] [1849, 2481]
OSJL
[397387.5, 3153427.5] [1849, 2481]
SC
[397387.5, 3153427.5] [1849, 2481]
DL
[397387.5, 3153427.5] [1849, 2481]
DZ
[397387.5, 3153427.5] [1849, 2481]
SC2
[397387.5, 3153427.5] [1849, 2481]
